In [ ]:
!pip install torch #installed -no need to run

In [ ]:
!git clone https://github.com/ultralytics/ultralytics.git # cloned -no need to run

In [ ]:
!pip install -r ultralytics/requirements.txt #requirements fulfilled -no need to run 

In [ ]:
!pip install ultralytics #done -no need to run

In [ ]:
#CLI option
#!yolo task=detect mode=predict model=yolov8x.pt source="" show=True

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
detection_model = YOLO("yolov8x.pt")

In [6]:
def locateObj(res,image):
    boxes=res[0].boxes
    box=boxes[0]
    x1=int(box.xyxy[0][0])
    y1=int(box.xyxy[0][1])
    x2=int(box.xyxy[0][2])
    y2=int(box.xyxy[0][3])
    img = cv2.imread(image)
    img_np = np.asarray(img)
    bounded_img= img_np[y1:y2,x1:x2]
    return bounded_img

In [48]:
image="/home/ai-05/Desktop/sdp/Custom-Dataset/Plastic/frame_000389.PNG"
results = detection_model.predict(source=image,show=True)


image 1/1 /home/ai-05/Desktop/sdp/Custom-Dataset/Plastic/frame_000389.PNG: 384x640 1 bottle, 26.7ms
Speed: 0.4ms preprocess, 26.7ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)


In [49]:
#len(results[0].boxes)=0 means no detection and > 1 means multiple objects detected
objs = len(results[0].boxes)
if(objs ==0 or objs>1):
    pass # to be replace by call to the classifier sending the entire captured image
elif(objs==1):
    bounded_img=locateObj(results,image)
    cv2.imshow("Bounded Frame",bounded_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    #pass bounded frame to the classifier model

In [52]:
!yolo export model=yolov8x.pt format='tflite'

/bin/bash: /home/ai-05/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2023-04-03 15:43:00.638626: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Ultralytics YOLOv8.0.59 🚀 Python-3.10.9 torch-2.0.0+cu117 CPU
YOLOv8x summary (fused): 268 layers, 68200608 parameters, 0 gradients, 257.8 GFLOPs

PyTorch: starting from yolov8x.pt with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (130.5 MB)

TensorFlow SavedModel: starting export with tensorflow 2.12.0...

ONNX: starting export with onnx 1.13.1...
============= Diagnostic Run torch.onnx.export version 2.0.0+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

ONNX: simplifying with onnxsim 0.4.19...
ONNX: export success ✅ 18.8s, saved as yolov8x.onnx (260.4 MB)

TensorFlow SavedModel: running 'onnx2tf -i yolov8x.onnx -o yolov8x_saved_model -nuo -

In [ ]:
import onnx
onnx_model = onnx.load("yolov8x.onnx")

from onnx_tf.backend import prepare
tf_rep = prepare(onnx_model)

tf_rep.export_graph("/home/ai-05/Desktop/sdp/")

import tensorflow as tf

model = tf.saved_model.load(tf_model_path)
model.trainable = False

input_tensor = tf.random.uniform([batch_size, channels, height, width])
out = model(**{'input': input_tensor})